# Rental Rates of Major Metro Markets
## Machine Learning Model based off data provided by Zillow.com

Zillow Observed Rent Index (ZORI): A smoothed measure of the typical observed market rate rent across a given region. ZORI is a repeat-rent index that is weighted to the rental housing stock to ensure representativeness across the entire market, not just those homes currently listed for-rent. The index is dollar-denominated by computing the mean of listed rents that fall into the 40th to 60th percentile range for all homes and apartments in a given region, which is once again weighted to reflect the rental housing stock. Details available in ZORI methodology.
https://www.zillow.com/research/methodology-zori-repeat-rent-27092/

In [130]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cluster import DBSCAN, KMeans
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report, silhouette_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [132]:
df = pd.read_csv('/content/drive/MyDrive/DS-CodingDojoColab/Project2/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv')
df.tail()

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
2258,61618,10004,8842,"New York, NY",2836.0,2863.0,2890.0,2916.0,2942.0,2968.0,2994.0,3020.0,3046.0,3072.0,3093.0,3114.0,3135.0,3154.0,3173.0,3192.0,3214.0,3236.0,3258.0,3268.0,3279.0,3290.0,3291.0,3292.0,3294.0,3289.0,3284.0,3280.0,3270.0,3260.0,3250.0,3247.0,3244.0,3241.0,3240.0,3240.0,...,3210.0,3207.0,3204.0,3202.0,3200.0,3203.0,3205.0,3208.0,3211.0,3214.0,3217.0,3218.0,3219.0,3219.0,3224.0,3229.0,3233.0,3240.0,3247.0,3254.0,3263.0,3273.0,3282.0,3290.0,3298.0,3306.0,3299.0,3293.0,3286.0,3247.0,3207.0,3168.0,3118.0,3068.0,3018.0,2965.0,2913.0,2860.0,2803.0,2747.0
2259,58622,2108,8986,"Boston, MA",2392.0,2393.0,2394.0,2395.0,2396.0,2398.0,2400.0,2402.0,2403.0,2405.0,2412.0,2418.0,2425.0,2436.0,2446.0,2457.0,2463.0,2468.0,2474.0,2478.0,2481.0,2485.0,2490.0,2495.0,2500.0,2503.0,2506.0,2509.0,2516.0,2522.0,2529.0,2539.0,2549.0,2559.0,2565.0,2572.0,...,2585.0,2582.0,2582.0,2582.0,2581.0,2586.0,2590.0,2594.0,2597.0,2599.0,2602.0,2607.0,2611.0,2616.0,2626.0,2636.0,2646.0,2657.0,2667.0,2678.0,2694.0,2710.0,2725.0,2745.0,2764.0,2783.0,2788.0,2793.0,2798.0,2787.0,2776.0,2765.0,2752.0,2739.0,2726.0,2711.0,2696.0,2681.0,2664.0,2647.0
2260,72525,33306,9047,"Miami-Fort Lauderdale, FL",1105.0,1121.0,1137.0,1153.0,1167.0,1182.0,NaN,1209.0,1222.0,1235.0,NaN,1250.0,1257.0,1258.0,1259.0,NaN,1255.0,1250.0,1244.0,NaN,NaN,1240.0,NaN,NaN,1252.0,1255.0,1258.0,1260.0,1265.0,1270.0,1275.0,1281.0,1287.0,NaN,NaN,1300.0,...,1318.0,1323.0,1327.0,1332.0,1336.0,1342.0,1348.0,1354.0,1361.0,1368.0,1374.0,1380.0,NaN,NaN,1400.0,1410.0,NaN,1432.0,1443.0,1455.0,1465.0,1474.0,1484.0,1488.0,1491.0,1495.0,1493.0,1491.0,1489.0,1491.0,NaN,1497.0,1500.0,1502.0,1505.0,1507.0,1510.0,1513.0,1516.0,NaN
2261,58624,2110,9469,"Boston, MA",4037.0,4030.0,4023.0,4016.0,NaN,4006.0,4001.0,NaN,4006.0,NaN,NaN,NaN,4015.0,NaN,4087.0,4124.0,4187.0,4250.0,4313.0,NaN,4356.0,NaN,4386.0,NaN,4403.0,4417.0,4430.0,4444.0,4467.0,4490.0,4513.0,4550.0,4586.0,4622.0,NaN,4666.0,...,NaN,4557.0,4539.0,4522.0,4504.0,4486.0,4469.0,4452.0,4456.0,4460.0,4463.0,4478.0,4492.0,4507.0,4518.0,4529.0,4540.0,4556.0,4572.0,4587.0,4606.0,4624.0,4642.0,4636.0,4629.0,4623.0,4601.0,4580.0,4558.0,4512.0,NaN,NaN,4366.0,4313.0,4260.0,4204.0,4149.0,4093.0,4037.0,3980.0
2262,66128,20004,9592,"Washington, DC",NaN,NaN,2297.0,2308.0,NaN,2329.0,NaN,2349.0,2359.0,NaN,NaN,NaN,2380.0,2377.0,2375.0,2372.0,2373.0,2373.0,2374.0,2375.0,2377.0,NaN,2377.0,2374.0,NaN,2379.0,2385.0,2391.0,2399.0,2408.0,2416.0,2424.0,2432.0,2440.0,2452.0,2465.0,...,2473.0,2465.0,2454.0,2443.0,2432.0,2423.0,2415.0,2406.0,2401.0,2397.0,2392.0,2394.0,2395.0,2396.0,2399.0,2401.0,2404.0,2405.0,2407.0,2408.0,2412.0,2417.0,2421.0,2425.0,2429.0,2434.0,2438.0,2441.0,2445.0,2444.0,2443.0,2441.0,2436.0,2431.0,2426.0,2420.0,2414.0,2408.0,2401.0,2394.0


I can already see in the last 5 rows alone that there are some NaN null values... let's get some basic info and find null values.

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2263 entries, 0 to 2262
Data columns (total 88 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RegionID    2263 non-null   int64  
 1   RegionName  2263 non-null   int64  
 2   SizeRank    2263 non-null   int64  
 3   MsaName     2263 non-null   object 
 4   2014-01     1726 non-null   float64
 5   2014-02     1785 non-null   float64
 6   2014-03     1740 non-null   float64
 7   2014-04     1961 non-null   float64
 8   2014-05     2030 non-null   float64
 9   2014-06     2094 non-null   float64
 10  2014-07     2129 non-null   float64
 11  2014-08     2119 non-null   float64
 12  2014-09     2113 non-null   float64
 13  2014-10     2062 non-null   float64
 14  2014-11     2022 non-null   float64
 15  2014-12     2027 non-null   float64
 16  2015-01     2066 non-null   float64
 17  2015-02     2060 non-null   float64
 18  2015-03     2144 non-null   float64
 19  2015-04     2146 non-null  

It appears that there are a lot of missing values for each month's rent reporting, so we're going to want to impute those.  Given that we generally know some value for a given location's rent before or after a missing month, it's a safe assumption we can fill the missing values with the median of before and after.

https://www.geeksforgeeks.org/python-pandas-dataframe-interpolate/



In [134]:
df.dtypes.value_counts()
df

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
0,61639,10025,1,"New York, NY",3025.0,3039.0,3054.0,3068.0,3083.0,3097.0,3112.0,3126.0,3140.0,3155.0,3169.0,3182.0,3196.0,3207.0,3217.0,3227.0,3233.0,3239.0,3244.0,3248.0,3252.0,3257.0,3260.0,3264.0,3268.0,3270.0,3272.0,3274.0,3276.0,3277.0,3279.0,3277.0,3276.0,3275.0,3275.0,3274.0,...,3267.0,3264.0,3263.0,3261.0,3260.0,3260.0,3260.0,3260.0,3262.0,3264.0,3267.0,3270.0,3274.0,3277.0,3281.0,3286.0,3290.0,3298.0,3305.0,3313.0,3324.0,3334.0,3345.0,3353.0,3361.0,3369.0,3361.0,3353.0,3346.0,3311.0,3277.0,3242.0,3201.0,3159.0,3117.0,3073.0,3029.0,2985.0,2938.0,2891.0
1,84654,60657,2,"Chicago, IL",1580.0,1585.0,1591.0,1596.0,1602.0,1607.0,1612.0,1618.0,1623.0,1628.0,1633.0,1638.0,1643.0,1648.0,1653.0,1657.0,1662.0,1666.0,1670.0,1675.0,1679.0,1684.0,1689.0,1694.0,1699.0,1703.0,1708.0,1713.0,1717.0,1722.0,1726.0,1730.0,1735.0,1739.0,1742.0,1745.0,...,1761.0,1762.0,1763.0,1765.0,1766.0,1766.0,1766.0,1767.0,1767.0,1767.0,1767.0,1768.0,1769.0,1770.0,1773.0,1776.0,1778.0,1783.0,1788.0,1793.0,1799.0,1806.0,1812.0,1817.0,1822.0,1827.0,1828.0,1829.0,1830.0,1825.0,1820.0,1816.0,1810.0,1803.0,1797.0,1791.0,1784.0,1777.0,1770.0,1762.0
2,61637,10023,3,"New York, NY",3156.0,3164.0,3172.0,3180.0,3188.0,3195.0,3203.0,3210.0,3218.0,3225.0,3231.0,3237.0,3242.0,3248.0,3254.0,3260.0,3267.0,3274.0,3281.0,3287.0,3294.0,3300.0,3305.0,3309.0,3314.0,3314.0,3313.0,3313.0,3309.0,3305.0,3301.0,3298.0,3295.0,3292.0,3293.0,3295.0,...,3304.0,3303.0,3300.0,3298.0,3296.0,3295.0,3295.0,3295.0,3299.0,3303.0,3306.0,3310.0,3314.0,3317.0,3323.0,3328.0,3333.0,3341.0,3350.0,3358.0,3366.0,3375.0,3384.0,3391.0,3397.0,3404.0,3397.0,3390.0,3383.0,3349.0,3316.0,3282.0,3241.0,3201.0,3160.0,3116.0,3073.0,3029.0,2983.0,2937.0
3,91982,77494,4,"Houston, TX",1778.0,1781.0,1784.0,1787.0,1790.0,1793.0,1795.0,1798.0,1801.0,1803.0,1805.0,1806.0,1808.0,1807.0,1807.0,1807.0,1805.0,1804.0,1802.0,1799.0,1796.0,1792.0,1787.0,1782.0,1777.0,1770.0,1764.0,1758.0,1751.0,1744.0,1738.0,1733.0,1729.0,1724.0,1722.0,1720.0,...,1735.0,1739.0,1742.0,1745.0,1747.0,1749.0,1750.0,1751.0,1751.0,1751.0,1751.0,1752.0,1753.0,1753.0,1755.0,1757.0,1759.0,1762.0,1764.0,1767.0,1769.0,1771.0,1773.0,1775.0,1776.0,1778.0,1779.0,1780.0,1781.0,1783.0,1785.0,1787.0,1791.0,1794.0,1797.0,1800.0,1804.0,1807.0,1810.0,1814.0
4,84616,60614,5,"Chicago, IL",1704.0,1709.0,1714.0,1720.0,1724.0,1729.0,1734.0,1738.0,1743.0,1747.0,1752.0,1756.0,1761.0,1766.0,1771.0,1776.0,1782.0,1787.0,1793.0,1801.0,1809.0,1818.0,1825.0,1833.0,1840.0,1846.0,1851.0,1857.0,1861.0,1864.0,1868.0,1871.0,1874.0,1876.0,1878.0,1880.0,...,1902.0,1905.0,1908.0,1910.0,1912.0,1914.0,1915.0,1917.0,1918.0,1919.0,1921.0,1923.0,1926.0,1928.0,1932.0,1935.0,1939.0,1945.0,1950.0,1956.0,1962.0,1969.0,1975.0,1979.0,1984.0,1988.0,1990.0,1991.0,1993.0,1986.0,1980.0,1973.0,1963.0,1953.0,1943.0,1931.0,1920.0,1909.0,1897.0,1885.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,61618,10004,8842,"New York, NY",2836.0,2863.0,2890.0,2916.0,2942.0,2968.0,2994.0,3020.0,3046.0,3072.0,3093.0,3114.0,3135.0,3154.0,3173.0,3192.0,3214.0,3236.0,3258.0,3268.0,3279.0,3290.0

In [143]:
df_sort = df.sort_values(by=['RegionName'], axis=0)
df_sort.head()

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
774,58477,1752,1303,"Boston, MA",1265.0,1269.0,1272.0,1276.0,1280.0,1283.0,1287.0,1291.0,1295.0,1298.0,1302.0,1306.0,1311.0,1315.0,1319.0,1323.0,1328.0,1333.0,1338.0,1344.0,1350.0,1357.0,1364.0,1371.0,1378.0,1384.0,1391.0,1397.0,1402.0,1408.0,1413.0,1418.0,1422.0,1427.0,1432.0,1436.0,...,1481.0,1487.0,1493.0,1500.0,1507.0,1515.0,1523.0,1531.0,1542.0,1552.0,1563.0,1573.0,1584.0,1595.0,1604.0,1613.0,1623.0,1631.0,1639.0,1647.0,1652.0,1656.0,1660.0,1661.0,1663.0,1664.0,1664.0,1665.0,1665.0,1666.0,1666.0,1667.0,1667.0,1668.0,1668.0,1668.0,1669.0,1669.0,1670.0,1671.0
1143,58481,1760,2041,"Boston, MA",1752.0,1750.0,1748.0,1746.0,1744.0,1741.0,1739.0,1737.0,1735.0,NaN,1734.0,1735.0,1736.0,1735.0,1735.0,1735.0,1734.0,1734.0,1734.0,1738.0,1742.0,1746.0,1752.0,1759.0,1765.0,1770.0,1774.0,1778.0,1776.0,1773.0,1771.0,1769.0,1768.0,1767.0,1769.0,1771.0,...,1810.0,1814.0,1819.0,1824.0,1829.0,1838.0,1847.0,1857.0,1869.0,1881.0,1893.0,1903.0,1913.0,1922.0,1926.0,1930.0,1934.0,1935.0,1935.0,1936.0,1939.0,1943.0,1947.0,1953.0,1960.0,1966.0,1971.0,1975.0,1979.0,1978.0,1977.0,1976.0,1973.0,1970.0,1966.0,1962.0,1957.0,1953.0,1947.0,1942.0
857,58489,1801,1445,"Boston, MA",1460.0,NaN,NaN,1520.0,1539.0,1558.0,1577.0,1597.0,1616.0,NaN,1651.0,1667.0,1682.0,NaN,1703.0,1713.0,1721.0,1730.0,1738.0,1747.0,1756.0,1765.0,1769.0,1773.0,1777.0,1782.0,1788.0,1793.0,1804.0,1815.0,1825.0,1835.0,1845.0,1856.0,1867.0,1879.0,...,1943.0,1948.0,1954.0,1959.0,1965.0,1973.0,1981.0,1989.0,1999.0,2009.0,2020.0,2029.0,2038.0,2047.0,2054.0,2061.0,2068.0,2073.0,2078.0,2083.0,2088.0,2092.0,2097.0,2100.0,2104.0,2108.0,2109.0,2111.0,2113.0,2108.0,2102.0,2097.0,2088.0,2079.0,2070.0,2060.0,2051.0,2041.0,2030.0,2020.0
1773,58517,1851,3728,"Boston, MA",995.0,1005.0,1014.0,1024.0,1033.0,1042.0,1051.0,1060.0,1069.0,1078.0,1083.0,1088.0,1093.0,NaN,1101.0,NaN,1112.0,1120.0,1127.0,1137.0,1146.0,1156.0,1167.0,1177.0,1187.0,1195.0,NaN,1212.0,1217.0,1222.0,1226.0,1229.0,1232.0,1235.0,NaN,1244.0,...,1283.0,1287.0,1294.0,1300.0,1306.0,1314.0,1323.0,1331.0,1339.0,1348.0,1357.0,1364.0,1371.0,1377.0,1381.0,1385.0,1389.0,1391.0,1393.0,1395.0,1397.0,1399.0,1401.0,1405.0,1409.0,1413.0,1416.0,1420.0,1423.0,1426.0,1428.0,1430.0,1433.0,1436.0,1439.0,1442.0,1444.0,1447.0,1450.0,1452.0
1178,58518,1852,2122,"Boston, MA",1065.0,NaN,1101.0,1118.0,1135.0,1152.0,1168.0,1184.0,1200.0,NaN,1226.0,1236.0,NaN,1252.0,1257.0,1262.0,NaN,1274.0,1279.0,1287.0,1295.0,1303.0,NaN,1319.0,1328.0,1335.0,1342.0,1349.0,1355.0,1360.0,1365.0,1371.0,1376.0,1381.0,1388.0,1394.0,...,1437.0,1442.0,1449.0,1456.0,1463.0,1474.0,1485.0,1495.0,1507.0,1518.0,1530.0,1540.0,1550.0,1560.0,1567.0,1573.0,1579.0,1583.0,1587.0,1590.0,1594.0,1597.0,1601.0,1606.0,1610.0,1615.0,1619.0,1624.0,1628.0,1633.0,1638.0,1643.0,1647.0,1650.0,1654.0,1657.0,1661.0,1664.0,1667.0,1671.0


In [137]:
# We'll rename RegionName to Zip Code since it's very confusing.  The number corresponds with the zip code.
df_sort.rename(columns = {'RegionName' : 'Zip Code'}, inplace=True)
df_sort.head()

,RegionID,Zip Code,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
774,58477,1752,1303,"Boston, MA",1265.0,1269.0,1272.0,1276.0,1280.0,1283.0,1287.0,1291.0,1295.0,1298.0,1302.0,1306.0,1311.0,1315.0,1319.0,1323.0,1328.0,1333.0,1338.0,1344.0,1350.0,1357.0,1364.0,1371.0,1378.0,1384.0,1391.0,1397.0,1402.0,1408.0,1413.0,1418.0,1422.0,1427.0,1432.0,1436.0,...,1481.0,1487.0,1493.0,1500.0,1507.0,1515.0,1523.0,1531.0,1542.0,1552.0,1563.0,1573.0,1584.0,1595.0,1604.0,1613.0,1623.0,1631.0,1639.0,1647.0,1652.0,1656.0,1660.0,1661.0,1663.0,1664.0,1664.0,1665.0,1665.0,1666.0,1666.0,1667.0,1667.0,1668.0,1668.0,1668.0,1669.0,1669.0,1670.0,1671.0
1143,58481,1760,2041,"Boston, MA",1752.0,1750.0,1748.0,1746.0,1744.0,1741.0,1739.0,1737.0,1735.0,NaN,1734.0,1735.0,1736.0,1735.0,1735.0,1735.0,1734.0,1734.0,1734.0,1738.0,1742.0,1746.0,1752.0,1759.0,1765.0,1770.0,1774.0,1778.0,1776.0,1773.0,1771.0,1769.0,1768.0,1767.0,1769.0,1771.0,...,1810.0,1814.0,1819.0,1824.0,1829.0,1838.0,1847.0,1857.0,1869.0,1881.0,1893.0,1903.0,1913.0,1922.0,1926.0,1930.0,1934.0,1935.0,1935.0,1936.0,1939.0,1943.0,1947.0,1953.0,1960.0,1966.0,1971.0,1975.0,1979.0,1978.0,1977.0,1976.0,1973.0,1970.0,1966.0,1962.0,1957.0,1953.0,1947.0,1942.0
857,58489,1801,1445,"Boston, MA",1460.0,NaN,NaN,1520.0,1539.0,1558.0,1577.0,1597.0,1616.0,NaN,1651.0,1667.0,1682.0,NaN,1703.0,1713.0,1721.0,1730.0,1738.0,1747.0,1756.0,1765.0,1769.0,1773.0,1777.0,1782.0,1788.0,1793.0,1804.0,1815.0,1825.0,1835.0,1845.0,1856.0,1867.0,1879.0,...,1943.0,1948.0,1954.0,1959.0,1965.0,1973.0,1981.0,1989.0,1999.0,2009.0,2020.0,2029.0,2038.0,2047.0,2054.0,2061.0,2068.0,2073.0,2078.0,2083.0,2088.0,2092.0,2097.0,2100.0,2104.0,2108.0,2109.0,2111.0,2113.0,2108.0,2102.0,2097.0,2088.0,2079.0,2070.0,2060.0,2051.0,2041.0,2030.0,2020.0
1773,58517,1851,3728,"Boston, MA",995.0,1005.0,1014.0,1024.0,1033.0,1042.0,1051.0,1060.0,1069.0,1078.0,1083.0,1088.0,1093.0,NaN,1101.0,NaN,1112.0,1120.0,1127.0,1137.0,1146.0,1156.0,1167.0,1177.0,1187.0,1195.0,NaN,1212.0,1217.0,1222.0,1226.0,1229.0,1232.0,1235.0,NaN,1244.0,...,1283.0,1287.0,1294.0,1300.0,1306.0,1314.0,1323.0,1331.0,1339.0,1348.0,1357.0,1364.0,1371.0,1377.0,1381.0,1385.0,1389.0,1391.0,1393.0,1395.0,1397.0,1399.0,1401.0,1405.0,1409.0,1413.0,1416.0,1420.0,1423.0,1426.0,1428.0,1430.0,1433.0,1436.0,1439.0,1442.0,1444.0,1447.0,1450.0,1452.0
1178,58518,1852,2122,"Boston, MA",1065.0,NaN,1101.0,1118.0,1135.0,1152.0,1168.0,1184.0,1200.0,NaN,1226.0,1236.0,NaN,1252.0,1257.0,1262.0,NaN,1274.0,1279.0,1287.0,1295.0,1303.0,NaN,1319.0,1328.0,1335.0,1342.0,1349.0,1355.0,1360.0,1365.0,1371.0,1376.0,1381.0,1388.0,1394.0,...,1437.0,1442.0,1449.0,1456.0,1463.0,1474.0,1485.0,1495.0,1507.0,1518.0,1530.0,1540.0,1550.0,1560.0,1567.0,1573.0,1579.0,1583.0,1587.0,1590.0,1594.0,1597.0,1601.0,1606.0,1610.0,1615.0,1619.0,1624.0,1628.0,1633.0,1638.0,1643.0,1647.0,1650.0,1654.0,1657.0,1661.0,1664.0,1667.0,1671.0


The Region ID and Size Rank are not really something we need at the moment, since we have a way to identify each row by Zip Code and MsaName, so we'll drop them out of the DF.

In [138]:
df_drop = df_sort.drop(['RegionID','SizeRank'], axis=1);
df_drop.head()

,Zip Code,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
774,1752,"Boston, MA",1265.0,1269.0,1272.0,1276.0,1280.0,1283.0,1287.0,1291.0,1295.0,1298.0,1302.0,1306.0,1311.0,1315.0,1319.0,1323.0,1328.0,1333.0,1338.0,1344.0,1350.0,1357.0,1364.0,1371.0,1378.0,1384.0,1391.0,1397.0,1402.0,1408.0,1413.0,1418.0,1422.0,1427.0,1432.0,1436.0,1440.0,1445.0,...,1481.0,1487.0,1493.0,1500.0,1507.0,1515.0,1523.0,1531.0,1542.0,1552.0,1563.0,1573.0,1584.0,1595.0,1604.0,1613.0,1623.0,1631.0,1639.0,1647.0,1652.0,1656.0,1660.0,1661.0,1663.0,1664.0,1664.0,1665.0,1665.0,1666.0,1666.0,1667.0,1667.0,1668.0,1668.0,1668.0,1669.0,1669.0,1670.0,1671.0
1143,1760,"Boston, MA",1752.0,1750.0,1748.0,1746.0,1744.0,1741.0,1739.0,1737.0,1735.0,NaN,1734.0,1735.0,1736.0,1735.0,1735.0,1735.0,1734.0,1734.0,1734.0,1738.0,1742.0,1746.0,1752.0,1759.0,1765.0,1770.0,1774.0,1778.0,1776.0,1773.0,1771.0,1769.0,1768.0,1767.0,1769.0,1771.0,1772.0,1775.0,...,1810.0,1814.0,1819.0,1824.0,1829.0,1838.0,1847.0,1857.0,1869.0,1881.0,1893.0,1903.0,1913.0,1922.0,1926.0,1930.0,1934.0,1935.0,1935.0,1936.0,1939.0,1943.0,1947.0,1953.0,1960.0,1966.0,1971.0,1975.0,1979.0,1978.0,1977.0,1976.0,1973.0,1970.0,1966.0,1962.0,1957.0,1953.0,1947.0,1942.0
857,1801,"Boston, MA",1460.0,NaN,NaN,1520.0,1539.0,1558.0,1577.0,1597.0,1616.0,NaN,1651.0,1667.0,1682.0,NaN,1703.0,1713.0,1721.0,1730.0,1738.0,1747.0,1756.0,1765.0,1769.0,1773.0,1777.0,1782.0,1788.0,1793.0,1804.0,1815.0,1825.0,1835.0,1845.0,1856.0,1867.0,1879.0,1890.0,1901.0,...,1943.0,1948.0,1954.0,1959.0,1965.0,1973.0,1981.0,1989.0,1999.0,2009.0,2020.0,2029.0,2038.0,2047.0,2054.0,2061.0,2068.0,2073.0,2078.0,2083.0,2088.0,2092.0,2097.0,2100.0,2104.0,2108.0,2109.0,2111.0,2113.0,2108.0,2102.0,2097.0,2088.0,2079.0,2070.0,2060.0,2051.0,2041.0,2030.0,2020.0
1773,1851,"Boston, MA",995.0,1005.0,1014.0,1024.0,1033.0,1042.0,1051.0,1060.0,1069.0,1078.0,1083.0,1088.0,1093.0,NaN,1101.0,NaN,1112.0,1120.0,1127.0,1137.0,1146.0,1156.0,1167.0,1177.0,1187.0,1195.0,NaN,1212.0,1217.0,1222.0,1226.0,1229.0,1232.0,1235.0,NaN,1244.0,1249.0,1254.0,...,1283.0,1287.0,1294.0,1300.0,1306.0,1314.0,1323.0,1331.0,1339.0,1348.0,1357.0,1364.0,1371.0,1377.0,1381.0,1385.0,1389.0,1391.0,1393.0,1395.0,1397.0,1399.0,1401.0,1405.0,1409.0,1413.0,1416.0,1420.0,1423.0,1426.0,1428.0,1430.0,1433.0,1436.0,1439.0,1442.0,1444.0,1447.0,1450.0,1452.0
1178,1852,"Boston, MA",1065.0,NaN,1101.0,1118.0,1135.0,1152.0,1168.0,1184.0,1200.0,NaN,1226.0,1236.0,NaN,1252.0,1257.0,1262.0,NaN,1274.0,1279.0,1287.0,1295.0,1303.0,NaN,1319.0,1328.0,1335.0,1342.0,1349.0,1355.0,1360.0,1365.0,1371.0,1376.0,1381.0,1388.0,1394.0,1401.0,1406.0,...,1437.0,1442.0,1449.0,1456.0,1463.0,1474.0,1485.0,1495.0,1507.0,1518.0,1530.0,1540.0,1550.0,1560.0,1567.0,1573.0,1579.0,1583.0,1587.0,1590.0,1594.0,1597.0,1601.0,1606.0,1610.0,1615.0,1619.0,1624.0,1628.0,1633.0,1638.0,1643.0,1647.0,1650.0,1654.0,1657.0,1661.0,1664.0,1667.0,1671.0


We'll make the zip code the index column now.

In [139]:
df_drop.set_index('Zip Code', inplace=True)
df_drop.head()

,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
Zip Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1752,"Boston, MA",1265.0,1269.0,1272.0,1276.0,1280.0,1283.0,1287.0,1291.0,1295.0,1298.0,1302.0,1306.0,1311.0,1315.0,1319.0,1323.0,1328.0,1333.0,1338.0,1344.0,1350.0,1357.0,1364.0,1371.0,1378.0,1384.0,1391.0,1397.0,1402.0,1408.0,1413.0,1418.0,1422.0,1427.0,1432.0,1436.0,1440.0,1445.0,1449.0,...,1481.0,1487.0,1493.0,1500.0,1507.0,1515.0,1523.0,1531.0,1542.0,1552.0,1563.0,1573.0,1584.0,1595.0,1604.0,1613.0,1623.0,1631.0,1639.0,1647.0,1652.0,1656.0,1660.0,1661.0,1663.0,1664.0,1664.0,1665.0,1665.0,1666.0,1666.0,1667.0,1667.0,1668.0,1668.0,1668.0,1669.0,1669.0,1670.0,1671.0
1760,"Boston, MA",1752.0,1750.0,1748.0,1746.0,1744.0,1741.0,1739.0,1737.0,1735.0,NaN,1734.0,1735.0,1736.0,1735.0,1735.0,1735.0,1734.0,1734.0,1734.0,1738.0,1742.0,1746.0,1752.0,1759.0,1765.0,1770.0,1774.0,1778.0,1776.0,1773.0,1771.0,1769.0,1768.0,1767.0,1769.0,1771.0,1772.0,1775.0,1778.0,...,1810.0,1814.0,1819.0,1824.0,1829.0,1838.0,1847.0,1857.0,1869.0,1881.0,1893.0,1903.0,1913.0,1922.0,1926.0,1930.0,1934.0,1935.0,1935.0,1936.0,1939.0,1943.0,1947.0,1953.0,1960.0,1966.0,1971.0,1975.0,1979.0,1978.0,1977.0,1976.0,1973.0,1970.0,1966.0,1962.0,1957.0,1953.0,1947.0,1942.0
1801,"Boston, MA",1460.0,NaN,NaN,1520.0,1539.0,1558.0,1577.0,1597.0,1616.0,NaN,1651.0,1667.0,1682.0,NaN,1703.0,1713.0,1721.0,1730.0,1738.0,1747.0,1756.0,1765.0,1769.0,1773.0,1777.0,1782.0,1788.0,1793.0,1804.0,1815.0,1825.0,1835.0,1845.0,1856.0,1867.0,1879.0,1890.0,1901.0,1911.0,...,1943.0,1948.0,1954.0,1959.0,1965.0,1973.0,1981.0,1989.0,1999.0,2009.0,2020.0,2029.0,2038.0,2047.0,2054.0,2061.0,2068.0,2073.0,2078.0,2083.0,2088.0,2092.0,2097.0,2100.0,2104.0,2108.0,2109.0,2111.0,2113.0,2108.0,2102.0,2097.0,2088.0,2079.0,2070.0,2060.0,2051.0,2041.0,2030.0,2020.0
1851,"Boston, MA",995.0,1005.0,1014.0,1024.0,1033.0,1042.0,1051.0,1060.0,1069.0,1078.0,1083.0,1088.0,1093.0,NaN,1101.0,NaN,1112.0,1120.0,1127.0,1137.0,1146.0,1156.0,1167.0,1177.0,1187.0,1195.0,NaN,1212.0,1217.0,1222.0,1226.0,1229.0,1232.0,1235.0,NaN,1244.0,1249.0,1254.0,1259.0,...,1283.0,1287.0,1294.0,1300.0,1306.0,1314.0,1323.0,1331.0,1339.0,1348.0,1357.0,1364.0,1371.0,1377.0,1381.0,1385.0,1389.0,1391.0,1393.0,1395.0,1397.0,1399.0,1401.0,1405.0,1409.0,1413.0,1416.0,1420.0,1423.0,1426.0,1428.0,1430.0,1433.0,1436.0,1439.0,1442.0,1444.0,1447.0,1450.0,1452.0
1852,"Boston, MA",1065.0,NaN,1101.0,1118.0,1135.0,1152.0,1168.0,1184.0,1200.0,NaN,1226.0,1236.0,NaN,1252.0,1257.0,1262.0,NaN,1274.0,1279.0,1287.0,1295.0,1303.0,NaN,1319.0,1328.0,1335.0,1342.0,1349.0,1355.0,1360.0,1365.0,1371.0,1376.0,1381.0,1388.0,1394.0,1401.0,1406.0,1411.0,...,1437.0,1442.0,1449.0,1456.0,1463.0,1474.0,1485.0,1495.0,1507.0,1518.0,1530.0,1540.0,1550.0,1560.0,1567.0,1573.0,1579.0,1583.0,1587.0,1590.0,1594.0,1597.0,1601.0,1606.0,1610.0,1615.0,1619.0,1624.0,1628.0,1633.0,1638.0,1643.0,1647.0,1650.0,1654.0,1657.0,1661.0,1664.0,1667.0,1671.0


In [153]:
# We'll transpose columns into rows in order to facilitate imputing the null values
df_t = df_drop.transpose()
df_t

Zip Code,1752,1760,1801,1851,1852,1960,1970,2026,2108,2109,2110,2113,2114,2115,2116,2118,2119,2120,2121,2122,2124,2125,2127,2128,2130,2131,2132,2135,2138,2139,2141,2145,2148,2155,2169,2184,2210,2215,2301,2360,...,98118,98119,98121,98122,98133,98144,98155,98168,98198,98199,98201,98208,98258,98270,98271,98275,98327,98373,98374,98375,98387,98391,98402,98404,98405,98406,98409,98444,98445,98446,98466,98498,98607,98661,98662,98671,98682,98685,99205,99208
MsaName,"Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA",...,"Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Spokane, WA","Spokane, WA"
2014-01,1265,1752,1460,995,1065,1382,NaN,NaN,2392,NaN,4037,2136,2415,2377,2500,2565,1568,3116,NaN,1509,NaN,NaN,2282,1710,2052,1550,1673,1890,2133,2447,2391,2001,1721,1909,1501,2070,3108,1973,1326,NaN,...,1086,NaN,1912,1449,NaN,1220,1422,NaN,1145,1288,884,1095,1513,1487,NaN,1334,1453,NaN,1296,1426,1420,1205,980,940,944,NaN,961,865,1333,1149,NaN,1009,1473,896,1089,NaN,1064,NaN,735,793
2014-02,1269,1750,NaN,1005,NaN,NaN,NaN,NaN,2393,2817,4030,2148,2415,2382,2499,2571,NaN,3124,1744,NaN,NaN,NaN,2287,1723,2059,1565,1686,1897,2146,2448,2389,2005,1723,1914,1509,2073,3131,1978,1327,NaN,...,1097,1329,1921,1458,1165,NaN,1443,NaN,NaN,1318,897,1105,1525,1501,NaN,1336,1459,1139,1309,1437,1428,1219,988,NaN,951,985,965,870,1343,NaN,1176,1017,NaN,906,1092,NaN,NaN,NaN,732,806
2014-03,1272,1748,NaN,1014,1101,NaN,1413,1637,2394,2822,4023,2159,NaN,2387,2498,2576,NaN,3133,NaN,NaN,NaN,NaN,2293,1736,2066,1581,1699,1905,2160,2448,2387,NaN,1724,1919,1518,2077,NaN,1983,1328,NaN,...,1109,1345,1930,1468,1176,NaN,1464,NaN,1162,1348,910,1114,1538,1515,1258,1339,1466,1148,1321,1448,1436,1233,996,953,959,994,969,NaN,1353,NaN,1177,NaN,NaN,916,1096,NaN,1089,1513,728,818
2014-04,1276,1746,1520,1024,1118,1410,1420,NaN,2395,2827,4016,2170,2415,2392,2497,2581,1719,3142,NaN,1518,NaN,1853,2298,1749,2073,1596,1713,1912,2173,2448,2385,2011,1726,1924,1526,2081,3179,1989,1329,1459,...,1120,NaN,1940,1478,1188,1325,1485,NaN,1171,1379,923,1123,1550,1529,1266,1341,1473,NaN,1334,1459,1444,1247,1004,960,966,NaN,974,881,1363,1180,1178,1033,1519,927,1099,NaN,1102,1524,725,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08,1668,1962,2060,1442,1657,1887,1979,2062,2711,2899,4204,2341,2519,2601,2638,2876,2711,3658,2590,2302,2272,2439,2817,2313,2420,2201,2167,2195,2437,2660,2690,2539,2086,2332,1986,2455,3260,2265,1829,2061,...,1814,1804,2283,1847,1753,2039,2074,NaN,1731,2157,1411,1641,2319,2285,1848,1761,2025,1766,2040,2186,2118,1858,1432,1511,1481,1709,1417,1355,2055,1800,1708,1632,2252,1409,1522,1693,1607,2116,1233,1430
2020-09,1669,1957,2051,1444,1661,1889,1990,2047,2696,2863,4149,2312,2482,2573,2617,2843,2719,3644,2590,2315,2272,2431,2805,2302,2408,2194,2166,2180,2413,2637,26

In [165]:
# We're going to use bfill to impute subsequent months' rental rates into prior missing values in each market
df_t.loc[:, :].fillna(method='bfill', inplace=True)
df_t

Zip Code,1752,1760,1801,1851,1852,1960,1970,2026,2108,2109,2110,2113,2114,2115,2116,2118,2119,2120,2121,2122,2124,2125,2127,2128,2130,2131,2132,2135,2138,2139,2141,2145,2148,2155,2169,2184,2210,2215,2301,2360,...,98118,98119,98121,98122,98133,98144,98155,98168,98198,98199,98201,98208,98258,98270,98271,98275,98327,98373,98374,98375,98387,98391,98402,98404,98405,98406,98409,98444,98445,98446,98466,98498,98607,98661,98662,98671,98682,98685,99205,99208
MsaName,"Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA",...,"Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Spokane, WA","Spokane, WA"
2014-01,1265,1752,1460,995,1065,1382,1413,1637,2392,2817,4037,2136,2415,2377,2500,2565,1568,3116,1744,1509,1672,1853,2282,1710,2052,1550,1673,1890,2133,2447,2391,2001,1721,1909,1501,2070,3108,1973,1326,1459,...,1086,1329,1912,1449,1165,1220,1422,1212,1145,1288,884,1095,1513,1487,1258,1334,1453,1139,1296,1426,1420,1205,980,940,944,985,961,865,1333,1149,1176,1009,1473,896,1089,1229,1064,1513,735,793
2014-02,1269,1750,1520,1005,1101,1410,1413,1637,2393,2817,4030,2148,2415,2382,2499,2571,1719,3124,1744,1518,1672,1853,2287,1723,2059,1565,1686,1897,2146,2448,2389,2005,1723,1914,1509,2073,3131,1978,1327,1459,...,1097,1329,1921,1458,1165,1325,1443,1212,1162,1318,897,1105,1525,1501,1258,1336,1459,1139,1309,1437,1428,1219,988,953,951,985,965,870,1343,1180,1176,1017,1519,906,1092,1229,1089,1513,732,806
2014-03,1272,1748,1520,1014,1101,1410,1413,1637,2394,2822,4023,2159,2415,2387,2498,2576,1719,3133,1783,1518,1672,1853,2293,1736,2066,1581,1699,1905,2160,2448,2387,2011,1724,1919,1518,2077,3179,1983,1328,1459,...,1109,1345,1930,1468,1176,1325,1464,1212,1162,1348,910,1114,1538,1515,1258,1339,1466,1148,1321,1448,1436,1233,996,953,959,994,969,881,1353,1180,1177,1033,1519,916,1096,1229,1089,1513,728,818
2014-04,1276,1746,1520,1024,1118,1410,1420,1669,2395,2827,4016,2170,2415,2392,2497,2581,1719,3142,1783,1518,1672,1853,2298,1749,2073,1596,1713,1912,2173,2448,2385,2011,1726,1924,1526,2081,3179,1989,1329,1459,...,1120,1376,1940,1478,1188,1325,1485,1212,1171,1379,923,1123,1550,1529,1266,1341,1473,1164,1334,1459,1444,1247,1004,960,966,1013,974,881,1363,1180,1178,1033,1519,927,1099,1229,1102,1524,725,868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08,1668,1962,2060,1442,1657,1887,1979,2062,2711,2899,4204,2341,2519,2601,2638,2876,2711,3658,2590,2302,2272,2439,2817,2313,2420,2201,2167,2195,2437,2660,2690,2539,2086,2332,1986,2455,3260,2265,1829,2061,...,1814,1804,2283,1847,1753,2039,2074,1764,1731,2157,1411,1641,2319,2285,1848,1761,2025,1766,2040,2186,2118,1858,1432,1511,1481,1709,1417,1355,2055,1800,1708,1632,2252,1409,1522,1693,1607,2116,1233,1430
2020-09,1669,1957,2051,1444,1661,1889,1990,2047,2696,2863,4149,2312,2482,2573,2617,2843,2719,3644,25

In [166]:
# We're going to use ffill to impute a handful of null values at the bottom of each column (most recent months), that don't have a value in front to pull from
df_t.loc[:, :].fillna(method='ffill', inplace=True)
df_t

Zip Code,1752,1760,1801,1851,1852,1960,1970,2026,2108,2109,2110,2113,2114,2115,2116,2118,2119,2120,2121,2122,2124,2125,2127,2128,2130,2131,2132,2135,2138,2139,2141,2145,2148,2155,2169,2184,2210,2215,2301,2360,...,98118,98119,98121,98122,98133,98144,98155,98168,98198,98199,98201,98208,98258,98270,98271,98275,98327,98373,98374,98375,98387,98391,98402,98404,98405,98406,98409,98444,98445,98446,98466,98498,98607,98661,98662,98671,98682,98685,99205,99208
MsaName,"Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA","Boston, MA",...,"Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Seattle, WA","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Portland, OR","Spokane, WA","Spokane, WA"
2014-01,1265,1752,1460,995,1065,1382,1413,1637,2392,2817,4037,2136,2415,2377,2500,2565,1568,3116,1744,1509,1672,1853,2282,1710,2052,1550,1673,1890,2133,2447,2391,2001,1721,1909,1501,2070,3108,1973,1326,1459,...,1086,1329,1912,1449,1165,1220,1422,1212,1145,1288,884,1095,1513,1487,1258,1334,1453,1139,1296,1426,1420,1205,980,940,944,985,961,865,1333,1149,1176,1009,1473,896,1089,1229,1064,1513,735,793
2014-02,1269,1750,1520,1005,1101,1410,1413,1637,2393,2817,4030,2148,2415,2382,2499,2571,1719,3124,1744,1518,1672,1853,2287,1723,2059,1565,1686,1897,2146,2448,2389,2005,1723,1914,1509,2073,3131,1978,1327,1459,...,1097,1329,1921,1458,1165,1325,1443,1212,1162,1318,897,1105,1525,1501,1258,1336,1459,1139,1309,1437,1428,1219,988,953,951,985,965,870,1343,1180,1176,1017,1519,906,1092,1229,1089,1513,732,806
2014-03,1272,1748,1520,1014,1101,1410,1413,1637,2394,2822,4023,2159,2415,2387,2498,2576,1719,3133,1783,1518,1672,1853,2293,1736,2066,1581,1699,1905,2160,2448,2387,2011,1724,1919,1518,2077,3179,1983,1328,1459,...,1109,1345,1930,1468,1176,1325,1464,1212,1162,1348,910,1114,1538,1515,1258,1339,1466,1148,1321,1448,1436,1233,996,953,959,994,969,881,1353,1180,1177,1033,1519,916,1096,1229,1089,1513,728,818
2014-04,1276,1746,1520,1024,1118,1410,1420,1669,2395,2827,4016,2170,2415,2392,2497,2581,1719,3142,1783,1518,1672,1853,2298,1749,2073,1596,1713,1912,2173,2448,2385,2011,1726,1924,1526,2081,3179,1989,1329,1459,...,1120,1376,1940,1478,1188,1325,1485,1212,1171,1379,923,1123,1550,1529,1266,1341,1473,1164,1334,1459,1444,1247,1004,960,966,1013,974,881,1363,1180,1178,1033,1519,927,1099,1229,1102,1524,725,868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08,1668,1962,2060,1442,1657,1887,1979,2062,2711,2899,4204,2341,2519,2601,2638,2876,2711,3658,2590,2302,2272,2439,2817,2313,2420,2201,2167,2195,2437,2660,2690,2539,2086,2332,1986,2455,3260,2265,1829,2061,...,1814,1804,2283,1847,1753,2039,2074,1764,1731,2157,1411,1641,2319,2285,1848,1761,2025,1766,2040,2186,2118,1858,1432,1511,1481,1709,1417,1355,2055,1800,1708,1632,2252,1409,1522,1693,1607,2116,1233,1430
2020-09,1669,1957,2051,1444,1661,1889,1990,2047,2696,2863,4149,2312,2482,2573,2617,2843,2719,3644,25

In [162]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, MsaName to 2020-12
Columns: 2263 entries, 1752 to 99208
dtypes: object(2263)
memory usage: 1.5+ MB


In [169]:
# Now that missing values have been imputed let's transpose back to the original df shape
rents = df_t.transpose()
rents

,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12
Zip Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1752,"Boston, MA",1265,1269,1272,1276,1280,1283,1287,1291,1295,1298,1302,1306,1311,1315,1319,1323,1328,1333,1338,1344,1350,1357,1364,1371,1378,1384,1391,1397,1402,1408,1413,1418,1422,1427,1432,1436,1440,1445,1449,...,1481,1487,1493,1500,1507,1515,1523,1531,1542,1552,1563,1573,1584,1595,1604,1613,1623,1631,1639,1647,1652,1656,1660,1661,1663,1664,1664,1665,1665,1666,1666,1667,1667,1668,1668,1668,1669,1669,1670,1671
1760,"Boston, MA",1752,1750,1748,1746,1744,1741,1739,1737,1735,1734,1734,1735,1736,1735,1735,1735,1734,1734,1734,1738,1742,1746,1752,1759,1765,1770,1774,1778,1776,1773,1771,1769,1768,1767,1769,1771,1772,1775,1778,...,1810,1814,1819,1824,1829,1838,1847,1857,1869,1881,1893,1903,1913,1922,1926,1930,1934,1935,1935,1936,1939,1943,1947,1953,1960,1966,1971,1975,1979,1978,1977,1976,1973,1970,1966,1962,1957,1953,1947,1942
1801,"Boston, MA",1460,1520,1520,1520,1539,1558,1577,1597,1616,1651,1651,1667,1682,1703,1703,1713,1721,1730,1738,1747,1756,1765,1769,1773,1777,1782,1788,1793,1804,1815,1825,1835,1845,1856,1867,1879,1890,1901,1911,...,1943,1948,1954,1959,1965,1973,1981,1989,1999,2009,2020,2029,2038,2047,2054,2061,2068,2073,2078,2083,2088,2092,2097,2100,2104,2108,2109,2111,2113,2108,2102,2097,2088,2079,2070,2060,2051,2041,2030,2020
1851,"Boston, MA",995,1005,1014,1024,1033,1042,1051,1060,1069,1078,1083,1088,1093,1101,1101,1112,1112,1120,1127,1137,1146,1156,1167,1177,1187,1195,1212,1212,1217,1222,1226,1229,1232,1235,1244,1244,1249,1254,1259,...,1283,1287,1294,1300,1306,1314,1323,1331,1339,1348,1357,1364,1371,1377,1381,1385,1389,1391,1393,1395,1397,1399,1401,1405,1409,1413,1416,1420,1423,1426,1428,1430,1433,1436,1439,1442,1444,1447,1450,1452
1852,"Boston, MA",1065,1101,1101,1118,1135,1152,1168,1184,1200,1226,1226,1236,1252,1252,1257,1262,1274,1274,1279,1287,1295,1303,1319,1319,1328,1335,1342,1349,1355,1360,1365,1371,1376,1381,1388,1394,1401,1406,1411,...,1437,1442,1449,1456,1463,1474,1485,1495,1507,1518,1530,1540,1550,1560,1567,1573,1579,1583,1587,1590,1594,1597,1601,1606,1610,1615,1619,1624,1628,1633,1638,1643,1647,1650,1654,1657,1661,1664,1667,1671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98671,"Portland, OR",1229,1229,1229,1229,1229,1240,1251,1262,1272,1283,1299,1299,1307,1316,1324,1333,1341,1349,1358,1367,1376,1385,1394,1404,1413,1433,1433,1442,1455,1468,1481,1497,1513,1530,1541,1552,1563,1574,1574,...,1563,1560,1561,1563,1564,1570,1576,1586,1586,1591,1595,1600,1605,1610,1614,1619,1624,1628,1632,1636,1640,1643,1646,1648,1651,1653,1655,1656,1658,1662,1667,1671,1676,1681,1687,1693,1698,1704,1710,1716
98682,"Portland, OR",1064,1089,1089,1102,1114,1126,1138,1150,1163,1175,1188,1201,1213,1223,1232,1241,1250,1259,1268,1277,1286,1295,1304,1312,1320,1328,1335,1343,1350,1358,1365,1372,1379,1385,1392,1398,1404,1409,1414,...,1442,1446,1451,1456,1461,1467,1473,1478,1484,1489,1495,1501,1507,1512,1519,1525,1531,1535,1540,1545,1549,1553,1557,1559,1561,1563,1564,1565,1565,1570,1575,1580,1586,1593,1600,1607,1614,1621,1628,1636
98685,"Portla

In [170]:
rents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2263 entries, 1752 to 99208
Data columns (total 85 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MsaName  2263 non-null   object
 1   2014-01  2263 non-null   object
 2   2014-02  2263 non-null   object
 3   2014-03  2263 non-null   object
 4   2014-04  2263 non-null   object
 5   2014-05  2263 non-null   object
 6   2014-06  2263 non-null   object
 7   2014-07  2263 non-null   object
 8   2014-08  2263 non-null   object
 9   2014-09  2263 non-null   object
 10  2014-10  2263 non-null   object
 11  2014-11  2263 non-null   object
 12  2014-12  2263 non-null   object
 13  2015-01  2263 non-null   object
 14  2015-02  2263 non-null   object
 15  2015-03  2263 non-null   object
 16  2015-04  2263 non-null   object
 17  2015-05  2263 non-null   object
 18  2015-06  2263 non-null   object
 19  2015-07  2263 non-null   object
 20  2015-08  2263 non-null   object
 21  2015-09  2263 non-null   object
 

Now we're good to go and ready to start visualizing our data!

# Visualizations